# Building a Transformer Based Model from Scratch

Video link: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [1]:
# set the dataset you want to train on

dataset_dir = '../data/tiny-shakespeare.txt'

In [2]:
# read the dataset and inspection

with open(dataset_dir, 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print('length of the dataset: ', len(text))

length of the dataset:  1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# building the vocabulary -- all the unique characters that occur in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# tokenization is only needed for words-models. This one is based only on characters, so char=token

In [7]:
# create a mapping from characters to integers -- models can only work with numbers

stoi = { ch:i for i,ch in enumerate(chars) }    # string to index
itos = { i:ch for i,ch in enumerate(chars) }    # index to string
encode = lambda s: [stoi[c] for c in s] # encoder: take a str, output a list of ints
decode = lambda l: ''.join([itos[c] for c in l]) # decoder: take a list of ints, outputs a str

print(encode('pepelin'))
print(decode(encode('pepelin')))


[54, 43, 54, 43, 50, 47, 52]
pepelin


In [ ]:
# lets encode the entire dataset and store it into a torch.tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# split up the data into train and validation sets

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [ ]:
# Transformer will receive chunks and batches

# these are the chunks the Transformer will receive

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when context is {context} the target must be {target}")

when context is tensor([18]) the target must be 47
when context is tensor([18, 47]) the target must be 56
when context is tensor([18, 47, 56]) the target must be 57
when context is tensor([18, 47, 56, 57]) the target must be 58
when context is tensor([18, 47, 56, 57, 58]) the target must be 1
when context is tensor([18, 47, 56, 57, 58,  1]) the target must be 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) the target must be 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target must be 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4  # how many sequences to process in parallel
block_size = 8  # what's the max context lenght for each prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

# these are 32 examples of it

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')
    

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 5

In [ ]:
print(xb)   # the input for the transformer